In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [2]:
input_file_name = '../data/processed/processed_2014_2018_final.pickle'
with open(input_file_name, 'rb') as input_file:
    flights = pickle.load(input_file)

In [3]:
flights = flights.loc[flights['ReachedDest'] == True, ['Month', 'Weekday', 'DepTimeBlk', 'ActArrDelay']]

In [4]:
flights.head()

,Month,Weekday,DepTimeBlk,ActArrDelay
0,1,3,0900-0959,-4.0
1,1,4,0900-0959,20.0
2,1,2,1100-1159,2.0
3,1,3,1100-1159,-7.0
4,1,4,1100-1159,-16.0


In [5]:
flights.shape

(29377684, 4)

In [6]:
ord_encoder = OrdinalEncoder()
flights['DepTimeBlk'] = ord_encoder.fit_transform(flights[['DepTimeBlk']]).astype('int')

In [7]:
bins = [-np.inf, -240.0, -180.0, -120.0, -90.0, -60.0, -45.0, -30.0, -15.0, 0, 15.0, 30.0, 45.0, 60.0, 90.0, 120.0,
        180.0, 360.0, 720.0, 1140.0, 2280.0, np.inf]
labels = np.arange(len(bins) - 1)

flights['ActArrDelay'] = pd.cut(flights['ActArrDelay'], bins = bins, labels = labels)

In [8]:
flights.head()

,Month,Weekday,DepTimeBlk,ActArrDelay
0,1,3,4,8
1,1,4,4,10
2,1,2,6,9
3,1,3,6,8
4,1,4,6,7


In [9]:
X_train, X_test, y_train, y_test = train_test_split(flights.iloc[:, :-1], flights.iloc[:, -1], test_size= 0.3, 
                                                    random_state = 1812, stratify = flights['ActArrDelay'])

In [10]:
train_sample_size = 100_000
train_sample_inds = X_train.sample(train_sample_size).index

X_train_sample = X_train.loc[train_sample_inds]
y_train_sample = y_train.loc[train_sample_inds]

In [11]:
test_sample_size = 100_000
test_sample_inds = X_test.sample(test_sample_size).index

X_test_sample = X_test.loc[test_sample_inds]
y_test_sample = y_test.loc[test_sample_inds]

In [12]:
std_scaler_features = StandardScaler()
rndForest_model = RandomForestClassifier()

In [19]:
steps = [('scalling', std_scaler_features), 
         ('random_forest', rndForest_model)]
pipeline_model = Pipeline(steps = steps)

In [37]:
param_grid = {'random_forest__n_estimators': np.linspace(100, 1000, 19, dtype='int'),
              'random_forest__max_depth': np.linspace(5, 50, 10, dtype='int'), 
              'random_forest__criterion': ['log_loss', 'gini', 'entropy']}

# gridCV = GridSearchCV(estimator = pipeline_model, param_grid=param_grid, scoring='f1_weighted', n_jobs=-1, verbose=10)
rndCV = RandomizedSearchCV(estimator = pipeline_model, param_distributions=param_grid, scoring='f1_macro', n_iter = 100, n_jobs=-1, verbose=10)

In [38]:
rndCV.fit(X_train_sample, y_train_sample)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 3/5; 2/100] START random_forest__criterion=log_loss, random_forest__max_depth=5, random_forest__n_estimators=400
[CV 5/5; 1/100] START random_forest__criterion=log_loss, random_forest__max_depth=40, random_forest__n_estimators=1000
[CV 1/5; 2/100] START random_forest__criterion=log_loss, random_forest__max_depth=5, random_forest__n_estimators=400
[CV 1/5; 1/100] START random_forest__criterion=log_loss, random_forest__max_depth=40, random_forest__n_estimators=1000
[CV 2/5; 1/100] START random_forest__criterion=log_loss, random_forest__max_depth=40, random_forest__n_estimators=1000
[CV 2/5; 2/100] START random_forest__criterion=log_loss, random_forest__max_depth=5, random_forest__n_estimators=400
[CV 4/5; 1/100] START random_forest__criterion=log_loss, random_forest__max_depth=40, random_forest__n_estimators=1000
[CV 3/5; 1/100] START random_forest__criterion=log_loss, random_forest__max_depth=40, random_forest__n_estimat

/opt/anaconda3/envs/springboard/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 4/5; 4/100] START random_forest__criterion=entropy, random_forest__max_depth=5, random_forest__n_estimators=750
[CV 2/5; 3/100] END random_forest__criterion=entropy, random_forest__max_depth=45, random_forest__n_estimators=800;, score=0.043 total time=  29.8s
[CV 3/5; 3/100] END random_forest__criterion=entropy, random_forest__max_depth=45, random_forest__n_estimators=800;, score=0.043 total time=  29.6s
[CV 5/5; 4/100] START random_forest__criterion=entropy, random_forest__max_depth=5, random_forest__n_estimators=750
[CV 1/5; 5/100] START random_forest__criterion=gini, random_forest__max_depth=30, random_forest__n_estimators=500
[CV 1/5; 4/100] END random_forest__criterion=entropy, random_forest__max_depth=5, random_forest__n_estimators=750;, score=0.036 total time=  17.2s
[CV 2/5; 5/100] START random_forest__criterion=gini, random_forest__max_depth=30, random_forest__n_estimators=500
[CV 2/5; 4/100] END random_forest__criterion=entropy, random_forest__max_depth=5, random_forest__

RandomizedSearchCV(estimator=Pipeline(steps=[('scalling', StandardScaler()),
                                             ('random_forest',
                                              RandomForestClassifier())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'random_forest__criterion': ['log_loss',
                                                                     'gini',
                                                                     'entropy'],
                                        'random_forest__max_depth': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]),
                                        'random_forest__n_estimators': array([ 100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  800,  850,  900,  950, 1000])},
                   scoring='f1_macro', verbose=10)

In [39]:
rndCV.best_score_

0.04340256885847296

In [40]:
rndCV.best_params_

{'random_forest__n_estimators': 950,
 'random_forest__max_depth': 25,
 'random_forest__criterion': 'entropy'}

In [45]:
pca_model = PCA()
pca_model.fit(X_train_sample)

PCA()

In [46]:
pca_model.explained_variance_

array([23.04109181, 11.56068764,  3.95162679])

In [48]:
pca_model_dd = PCA(n_components=2)
X_transformed = pca_model_dd.fit_transform(X_train_sample)

In [49]:
X_transformed

array([[-5.04465755, -1.54625391],
       [-0.05326244, -4.57269958],
       [-5.04296192, -2.54746144],
       ...,
       [-4.01263053,  3.44740445],
       [-4.04130192, -3.55372201],
       [-0.02122551,  4.4296209 ]])

In [52]:
pca_dd = PCA()
rndForest_pca = RandomForestClassifier()

param_grid_pca = {'rndForestCls__n_estimators': np.linspace(100, 1000, 19, dtype='int'),
                  'rndForestCls__max_depth': np.linspace(5, 50, 10, dtype='int'), 
                  'rndForestCls__criterion': ['log_loss', 'gini', 'entropy'], 
                  'pca__n_components': [1, 2]}

pca_steps = [('pca', pca_dd), 
             ('rndForestCls', rndForest_pca)]

pl_pca = Pipeline(steps=pca_steps)

rndCV_pca = RandomizedSearchCV(estimator = pl_pca, param_distributions=param_grid_pca, scoring='f1_macro', n_iter = 100, n_jobs=-1, verbose=10)
rndCV_pca.fit(X_train_sample, y_train_sample)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5; 1/100] START pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=20, rndForestCls__n_estimators=450
[CV 1/5; 1/100] START pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=20, rndForestCls__n_estimators=450
[CV 3/5; 1/100] START pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=20, rndForestCls__n_estimators=450
[CV 4/5; 1/100] START pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=20, rndForestCls__n_estimators=450
[CV 1/5; 2/100] START pca__n_components=2, rndForestCls__criterion=log_loss, rndForestCls__max_depth=25, rndForestCls__n_estimators=300
[CV 2/5; 2/100] START pca__n_components=2, rndForestCls__criterion=log_loss, rndForestCls__max_depth=25, rndForestCls__n_estimators=300
[CV 5/5; 1/100] START pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=20, rndForestCls__n_estimators=450
[CV 3

/opt/anaconda3/envs/springboard/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 2/5; 35/100] START pca__n_components=1, rndForestCls__criterion=log_loss, rndForestCls__max_depth=50, rndForestCls__n_estimators=700
[CV 2/5; 34/100] END pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=5, rndForestCls__n_estimators=900;, score=0.036 total time=  39.2s
[CV 3/5; 35/100] START pca__n_components=1, rndForestCls__criterion=log_loss, rndForestCls__max_depth=50, rndForestCls__n_estimators=700
[CV 3/5; 34/100] END pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=5, rndForestCls__n_estimators=900;, score=0.036 total time=  38.7s
[CV 4/5; 35/100] START pca__n_components=1, rndForestCls__criterion=log_loss, rndForestCls__max_depth=50, rndForestCls__n_estimators=700
[CV 4/5; 33/100] END pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=40, rndForestCls__n_estimators=1000;, score=0.044 total time= 1.3min
[CV 5/5; 33/100] END pca__n_components=1, rndForestCls__criterion=gini, rndForestCls__max_depth=40

RandomizedSearchCV(estimator=Pipeline(steps=[('pca', PCA()),
                                             ('rndForestCls',
                                              RandomForestClassifier())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'pca__n_components': [1, 2],
                                        'rndForestCls__criterion': ['log_loss',
                                                                    'gini',
                                                                    'entropy'],
                                        'rndForestCls__max_depth': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]),
                                        'rndForestCls__n_estimators': array([ 100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  800,  850,  900,  950, 1000])},
                   scoring='f1_macro', verbose=10)

In [53]:
rndCV_pca.best_score_

0.04346549714309127

In [54]:
rndCV_pca.best_params_

{'rndForestCls__n_estimators': 100,
 'rndForestCls__max_depth': 30,
 'rndForestCls__criterion': 'entropy',
 'pca__n_components': 2}

In [69]:
train_sample_size = 20_000_000
train_sample_inds = X_train.sample(train_sample_size).index

X_train_sample = X_train.loc[train_sample_inds]
y_train_sample = y_train.loc[train_sample_inds]

test_sample_size = 1_000_000
test_sample_inds = X_test.sample(test_sample_size).index

X_test_sample = X_test.loc[test_sample_inds]
y_test_sample = y_test.loc[test_sample_inds]

In [70]:
pca_dd = PCA(n_components= 2)
rndForest_pca = RandomForestClassifier(n_estimators= 100, max_depth= 30, criterion= 'entropy', n_jobs=-1, verbose=10)

pca_steps = [('pca', pca_dd), 
             ('rndForestCls', rndForest_pca)]

pl_pca = Pipeline(steps=pca_steps)
pl_pca.fit(X_train_sample, y_train_sample)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 98.7min


building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 205.2min


...

building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100


[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 1068.0min


building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed: 1182.0min remaining: 49.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 1182.2min finished


Pipeline(steps=[('pca', PCA(n_components=2)),
                ('rndForestCls',
                 RandomForestClassifier(criterion='entropy', max_depth=30,
                                        n_jobs=-1, verbose=10))])

In [71]:
from sklearn.metrics import f1_score

f1_score(pl_pca.predict(X_train_sample), y_train_sample, average='macro')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  4.7min
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  7.9min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 11.6min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 15.7min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed: 20.5min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed: 25.4min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed: 31.2min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed: 37.0min
[Parallel(n_jobs=8)]: Done  96 out of 100 | elapsed: 43.1min remaining:  1.8min
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed: 44.8min finished


0.029154951950550723

In [72]:
f1_score(pl_pca.predict(X_test_sample), y_test_sample, average='macro')

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done  96 out of 100 | elapsed:    2.8s remaining:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.9s finished


0.03434970422424146